In [1]:
import azureml.core

print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

You are currently using version 1.0.23 of the Azure ML SDK

##### Read the workspace configuration file and create a reference object to the workspace

In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config("/dbfs/tmp/config.json")
print("Workspace name: " + ws.name, 
      "Azure region: " + ws.location,
      "Resource group: " + ws.resource_group, sep = "\n")

Found the config file in: /dbfs/tmp/config.json
Workspace name: ML-Service-Workspace
Azure region: eastus
Resource group: ML-Service-RG

##### Copy serialized model file from DBFS to local FS

In [5]:
#NOTE: Need to copy model from DBFS to local FS, as Azure ML service deployment always gets the model from the current working dir
import os

base_path = "/tmp/rf_model/"
model_name = "rf_model"

model_local = "file:" + os.getcwd() + "/" + model_name
dbutils.fs.cp(base_path, model_local, True)

local_path = os.getcwd() + "/" + model_name

In [6]:
%sh ls -la /databricks/driver/rf_model/rf_model.mllib

total 16
drwxr-xr-x 4 root root 4096 Apr 19 12:49 .
drwxr-xr-x 3 root root 4096 Apr 19 12:49 ..
drwxr-xr-x 2 root root 4096 Apr 19 12:49 metadata
drwxr-xr-x 5 root root 4096 Apr 19 12:49 stages

##### Register the model on Azure ML service

In [8]:
#Register the model
from azureml.core.model import Model

mymodel = Model.register(model_path = model_name + "/rf_model.mllib", # local path where the model file was copied to
                         model_name = model_name, # this is the name to be used to register the model on Azure ML service
                         description = "MLlib Randomforest Model",
                         workspace = ws)

print(mymodel.name, mymodel.description, mymodel.version)

Registering model rf_model
rf_model MLlib Randomforest Model 16

##### Create a model scoring script and save it to local FS

In [10]:
score_sparkml = """

import json
import pickle
import pyspark
from azureml.core.model import Model
from pyspark.ml import PipelineModel
from pyspark.ml.feature import IndexToString

def init():
    # One-time initialization of PySpark and predictive model

    global trainedModel
    global spark

    spark = pyspark.sql.SparkSession.builder.appName("MLlib Randomforest Model Operationalization").getOrCreate()
    model_name = "{model_name}" #interpolated
    model_path = Model.get_model_path(model_name)
    trainedModel = PipelineModel.load(model_path)
    
def run(input_json):
    if isinstance(trainedModel, Exception):
        return json.dumps({{"trainedModel":str(trainedModel)}})
      
    try:
        sc = spark.sparkContext
        input_list = [json.loads(input_json)]
        input_rdd = sc.parallelize(input_list)
        input_df = spark.read.json(input_rdd)
    
        # Compute prediction
        prediction = trainedModel.transform(input_df)
        idx_to_string = IndexToString(inputCol="prediction", outputCol="classe_predicted", labels=trainedModel.stages[-3].labels)
        prediction = idx_to_string.transform(prediction)
        
        #result = prediction.first().prediction
        predictions = prediction.collect()

        #Get each scored result
        preds = [str(x["classe_predicted"]) for x in predictions]
        result = ",".join(preds)
        # you can return any data type as long as it is JSON-serializable
        return json.dumps({{"result":result}})        
    except Exception as e:
        result = str(e)
        return json.dumps({{"error":result}})
    
""".format(model_name=model_name)

exec(score_sparkml)

with open("score_sparkml.py", "w") as file:
    file.write(score_sparkml)

##### Create a deployment configuration file, optionally listing any dependent packages

In [12]:
from azureml.core.conda_dependencies import CondaDependencies 

myacienv = CondaDependencies.create(conda_packages=['scikit-learn','numpy','pandas']) #showing how to add libs as an example - not needed for this model.

with open("mydeployenv.yml","w") as f:
    f.write(myacienv.serialize_to_string())

##### Create deployment configuration

In [14]:
#Deployment configuration
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 2, 
    tags = {'name':'Databricks Azure ML ACI'}, 
    description = 'ACI to serve ADB Randomforest model.')

##### Create an image to host the model scoring script and deploy the web service

In [16]:
#This will take 10-15 minutes to finish

service_name = "aciadbws"
runtime = "spark-py" 
driver_file = "score_sparkml.py"
my_conda_file = "mydeployenv.yml"

# image creation
from azureml.core.image import ContainerImage
myimage_config = ContainerImage.image_configuration(execution_script = driver_file,
                                                    runtime = runtime,
                                                    conda_file = my_conda_file)

# Webservice creation
myservice = Webservice.deploy_from_model(workspace=ws,
                                         name=service_name,
                                         deployment_config = myaci_config,
                                         models = [mymodel],
                                         image_config = myimage_config)

myservice.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image aciadbws:13, operation "Succeeded"
Creating service
Running...................................................................
SucceededACI service creation operation finished, operation "Succeeded"

In [17]:
print(myservice.state)

Healthy

In [18]:
print(myservice.get_logs())

2019-04-19T13:00:17,481872480+00:00 - gunicorn/run 
2019-04-19T13:00:17,481930882+00:00 - iot-server/run 
2019-04-19T13:00:17,482257694+00:00 - rsyslog/run 
2019-04-19T13:00:17,482871016+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-04-19T13:00:17,591309859+00:00 - iot-server/finish 1 0
2019-04-19T13:00:17,592282094+00:00 - Exit code 1 is normal. Not restarting iot-server.
{"timestamp": "2019-04-19T13:00:18.773435Z", "message": "Starting gunicorn 19.6.0", "host": "wk-caas-788a4d908f1542c5a1b02b2f290c532d-d653029852df100957b508", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Starting gunicorn %s", "stack_info": null}
{"timestamp": "2019-04-19T13:00:18.774210Z", "message": "Listening at: http://127.0.0.1:9090 (12)", "host": "wk-caas-788a4d908f1542c5a1b02b2f290c532d-d653029852df100957b508", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Listening at: %s (%s)", "stack_info": null}
{"timestamp": "2019-04-19T13:00:18.774319Z", "message": "Using worker: sync", "host": "wk-caas-788a4d908f1542c5a1b02b2f290c532d-d653029852df100957b508", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Using worker: %s", "stack_info": null}
{"timestamp": "2019-04-19T13:00:18.774925Z", "message": "worker timeout is set to 300", "host": "wk-caas-788a4d908f1542c5a1b02b2f290c532d-d653029852df100957b508", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "stack_info": null}
{"timestamp": "2019-04-19T13:00:18.776381Z", "message": "Booting worker with pid: 51", "host": "wk-caas-788a4d908f1542c5a1b02b2f290c532d-d653029852df100957b508", "path": "/home/mmlspark/lib/conda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.error", "msg": "Booting worker with pid: %s", "stack_info": null}
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/home/mmlspark/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.microsoft.ml.spark#mmlspark_2.11 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-17cad6b6-8863-4b4d-a429-65570a978795;1.0
	confs: [default]
	found com.microsoft.ml.spark#mmlspark_2.11;0.15 in spark-list
	found io.spray#spray-json_2.11;1.3.2 in central
	found com.microsoft.cntk#cntk;2.4 in central
	found org.openpnp#opencv;3.2.0-1 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents#httpclient;4.5.6 in central
	found org.apache.httpcomponents#httpcore;4.4.10 in central
	found commons-logging#commons-logging;1.2 in central
	found commons-codec#commons-codec;1.10 in central
	found com.microsoft.ml.lightgbm#lightgbmlib;2.1.250 in central
:: resolution report :: resolve 610ms :: artifacts dl 10ms
	:: modules in use:
	com.jcraft#jsch;0.1.54 from central in [default]
	com.microsoft.cntk#cntk;2.4 from central in [default]
	com.microsoft.ml.lightgbm#lightgbmlib;2.1.250 from central in [default]
	com.microsoft.ml.spark#mmlspark_2.11;0.15 from spark-list in [default]
	commons-codec#commons-codec;1.10 from central in [default]
	commons-logging#commons-logging;1.2 from central in [default]
	io.spray#spray-json_2.11;1.3.2 from central in [default]
	org.apache.httpcomponents#httpclient;4.5.6 from central in [default]
	org.apache.httpcomponents#httpcore;4.4.10